***Sistemin saglikli calismasi icin GPU olarak Tesla T4(Colab)kullanildi, train islemleri bu veya ortalama RTX veya GTX(yalnizca en ust kartlarindan) serisi kartlar kullanilabilir.***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

DATA_PATH = "/content/drive/MyDrive/ortuk_zorbalik_v2/dataset.json"
CKPT_PATH = "/content/drive/MyDrive/ortuk_zorbalik_v2/checkpoints"
FINAL_MODEL_PATH = "/content/drive/MyDrive/ortuk_zorbalik_v2/model_final"

os.makedirs(CKPT_PATH, exist_ok=True)
os.makedirs(FINAL_MODEL_PATH, exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip uninstall -y transformers
!pip install transformers==4.57.3 datasets accelerate evaluate scikit-learn -q

import sys
if "transformers" in sys.modules:
    del sys.modules["transformers"]


Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, BertConfig
import evaluate
import numpy as np
import torch
import torch.nn as nn

dataset = load_dataset("json", data_files=DATA_PATH)

def transform(example):
    return {"text": example["input"], "label": example["output"]}

dataset = dataset.map(transform)


In [ ]:
labels = sorted(list(set(dataset["train"]["label"])))
label2id = {l:i for i,l in enumerate(labels)}
id2label = {i:l for l,i in label2id.items()}

def encode_label(example):
    example["label"] = label2id[example["label"]]
    if "labels" in example:
        del example["labels"]
    return example

dataset = dataset.map(encode_label)
num_labels = len(labels)

dataset = dataset["train"].train_test_split(test_size=0.1, shuffle=True, seed=42)
train_dataset = dataset["train"]
valid_dataset = dataset["test"]


In [ ]:
model_name = "dbmdz/bert-base-turkish-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=128)

train_tokenized = train_dataset.map(tokenize, batched=True)
valid_tokenized = valid_dataset.map(tokenize, batched=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import AutoModelForSequenceClassification, BertConfig, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

model_name = "dbmdz/bert-base-turkish-uncased"

config = BertConfig.from_pretrained(model_name)
config.num_labels = num_labels  # Senin label sayın

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=config,
    quantization_config=bnb_config,
    device_map="auto"
)


lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "value"],  # BERT için genelde bu modüller
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, lora_config)

# Freeze işlemi
for name, param in model.named_parameters():
    if "encoder.layer." in name:
        layer_num = int(name.split("encoder.layer.")[1].split(".")[0])
        if layer_num < 8:
            param.requires_grad = False


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class TrainerWithLabelSmoothing(Trainer):
    def __init__(self, *args, smoothing=0.1, **kwargs):
        super().__init__(*args, **kwargs)
        self.smoothing = smoothing
        self.criterion = nn.CrossEntropyLoss(label_smoothing=smoothing)

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = self.criterion(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [ ]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=config,
    quantization_config=bnb_config,
    device_map="auto"
)

model = get_peft_model(model, lora_config)

for name, param in model.named_parameters():
    if "encoder.layer." in name:
        layer_num = int(name.split("encoder.layer.")[1].split(".")[0])
        if layer_num < 8:
            param.requires_grad = False


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir=CKPT_PATH,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    logging_steps=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    learning_rate=3e-4,
    weight_decay=0.1,
    warmup_ratio=0.1,
    fp16=False,
    bf16=False,
    load_best_model_at_end=True,
    save_total_limit=3,
    gradient_checkpointing=True,
    optim="adamw_torch",
    report_to=[]
)

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = np.mean(preds == labels)
    return {"accuracy": acc}


In [ ]:
# LoRA classifier modüllerini FP32'ye çevir
for name, param in model.named_parameters():
    if "modules_to_save" in name:
        param.data = param.data.to(torch.float32)
        param.requires_grad = True
        if param.grad is not None:
            param.grad = None

print("modules_to_save parametreleri FP32'ye çevrildi.")


modules_to_save parametreleri FP32'ye çevrildi.


In [ ]:
trainer = TrainerWithLabelSmoothing(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=valid_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()


/tmp/ipython-input-599064265.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TrainerWithLabelSmoothing.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Step,Training Loss,Validation Loss,Accuracy
100,1.230600,1.145846,0.570667
200,0.993600,0.974945,0.628000
300,0.920800,0.857388,0.732667
400,0.837500,0.778757,0.852667
500,0.781000,0.738709,0.866000
600,0.754100,0.710378,0.886000
700,0.746100,0.696341,0.910000
800,0.732800,0.687228,0.895333


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have req

TrainOutput(global_step=844, training_loss=0.8889546416947062, metrics={'train_runtime': 310.8202, 'train_samples_per_second': 43.433, 'train_steps_per_second': 2.715, 'total_flos': 181630087236000.0, 'train_loss': 0.8889546416947062, 'epoch': 1.0})

In [ ]:
model.save_pretrained(FINAL_MODEL_PATH)
tokenizer.save_pretrained(FINAL_MODEL_PATH)

print(f"Model ve tokenizer '{FINAL_MODEL_PATH}' klasörüne kaydedildi.")


Model ve tokenizer '/content/drive/MyDrive/ortuk_zorbalik_v2/model_final' klasörüne kaydedildi.


***Buradan sonrasindaki kodlari calistirin, olmadan sistem calismayacaktir.***

In [ ]:
!pip show transformers bitsandbytes peft


In [ ]:
!rm -f ~/.cache/huggingface/accelerate/default_config.yaml
